---
title:  ID identification
subtitle: limited feature extraction / anomaly detection
---

In [1]:
#| default_exp core/detection

In [ ]:
#| export
from datetime import timedelta
import polars as pl

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils.basic import *

from multipledispatch import dispatch

The first index is $$ \frac{\sigma(B)}{Max(\sigma(B_-),\sigma(B_+))} $$
The second index is $$ \frac{\sigma(B_- + B_+)} {\sigma(B_-) + \sigma(B_+)} $$
The ﬁrst two conditions guarantee that the ﬁeld changes of the IDs identiﬁed are large enough to be distinguished from the stochastic ﬂuctuations on magnetic ﬁelds, while the third is a supplementary condition toreduce the uncertainty of recognition.

third index (relative field jump) is $$ \frac{| \Delta \vec{B} |}{|B_{bg}|} $$ a supplementary condition to reduce the uncertainty of recognition

In [ ]:
#| export
from ids_finder.utils.basic import _expand_selectors

In [ ]:
#| export
# some helper functions
def pl_format_time(df: pl.LazyFrame | pl.DataFrame, tau: timedelta):
    return df.with_columns(
        tstart=pl.col("time"),
        tstop=(pl.col("time") + tau).dt.cast_time_unit("ns"),
        time=(pl.col("time") + tau / 2).dt.cast_time_unit("ns"),
    )


def pl_dvec(columns, *more_columns):
    all_columns = _expand_selectors(columns, *more_columns)
    return [
        (pl.col(column).first() - pl.col(column).last()).alias(f"d{column}_vec")
        for column in all_columns
    ]

In [ ]:
# | export
def compute_std(
    df: pl.DataFrame | pl.LazyFrame, tau, b_cols=["BX", "BY", "BZ"]
) -> pl.DataFrame:
    b_std_cols = [col_name + "_std" for col_name in b_cols]

    std_df = (
        df.group_by_dynamic("time", every=tau / 2, period=tau)
        .agg(
            pl.count(),
            pl.col(b_cols).std(ddof=0).map_alias(lambda col_name: col_name + "_std"),
        )
        .with_columns(
            pl_norm(b_std_cols).alias("B_std"),
        )
        .drop(b_std_cols)
    )
    return std_df


def compute_combinded_std(
    df: pl.DataFrame | pl.LazyFrame, tau, cols
) -> pl.DataFrame:
    combined_std_cols = [col_name + "_combined_std" for col_name in cols]
    offsets = [0 * tau, tau / 2]
    combined_std_dfs = []
    for offset in offsets:
        truncated_df = df.select(
            (pl.col("time") - offset).dt.truncate(tau, offset=offset).alias("time"),
            pl.col(cols),
        )

        prev_df = truncated_df.select(
            (pl.col("time") + tau).dt.cast_time_unit("ns"),
            pl.col(cols),
        )

        next_df = truncated_df.select(
            (pl.col("time") - tau).dt.cast_time_unit("ns"),
            pl.col(cols),
        )

        temp_combined_std_df = (
            pl.concat([prev_df, next_df])
            .group_by("time")
            .agg(
                pl.col(cols)
                .std(ddof=0)
                .map_alias(lambda col_name: col_name + "_combined_std"),
            )
            .with_columns(pl_norm(combined_std_cols).alias("B_combined_std"))
            .drop(combined_std_cols)
            .sort("time")
        )

        combined_std_dfs.append(temp_combined_std_df)

    combined_std_df = pl.concat(combined_std_dfs)
    return combined_std_df

## Index of the standard deviation

In [ ]:
#| export
@dispatch(pl.LazyFrame, object)
def compute_index_std(df: pl.LazyFrame, tau, join_strategy="inner"):  # noqa: F811
    """
    Compute the standard deviation index based on the given DataFrame and tau value.

    Parameters
    ----------
    - df (pl.LazyFrame): The input DataFrame.
    - tau (int): The time interval value.

    Returns
    -------
    - pl.LazyFrame: DataFrame with calculated 'index_std' column.

    Examples
    --------
    >>> index_std_df = compute_index_std_pl(df, tau)
    >>> index_std_df

    Notes
    -----
    Simply shift to calculate index_std would not work correctly if data is missing, like `std_next = pl.col("B_std").shift(-2)`.

    """

    if isinstance(tau, (int, float)):
        tau = timedelta(seconds=tau)

    if "B_std" in df.columns:
        std_df = df
    else:
        # Compute standard deviations
        std_df = compute_std(df, tau)

    # Calculate the standard deviation index
    prev_std_df = std_df.select(
        (pl.col("time") + tau).dt.cast_time_unit("ns"),
        pl.col("B_std").alias("B_std_prev"),
        pl.col("count").alias("count_prev"),
    )

    next_std_df = std_df.select(
        (pl.col("time") - tau).dt.cast_time_unit("ns"),
        pl.col("B_std").alias("B_std_next"),
        pl.col("count").alias("count_next")
    )

    index_std_df = (
        std_df.join(prev_std_df, on="time", how=join_strategy)
        .join(next_std_df, on="time", how=join_strategy)
        .with_columns(
            (pl.col("B_std") / (pl.max_horizontal("B_std_prev", "B_std_next"))).alias(
                "index_std"
            )
        )
    )
    return index_std_df

## Index of fluctuation

## Index of the relative field jump

In [ ]:
#| export
def compute_index_diff(
    df: pl.DataFrame, 
    tau: timedelta,
    cols
    ):
    db_cols = ["d" + col + "_vec" for col in cols]

    index_diff = (
        df.with_columns(pl_norm(cols).alias("B"))
        .group_by_dynamic("time", every=tau / 2, period=tau)
        .agg(
            pl.col("B").mean().alias("B_mean"),
            *pl_dvec(cols),
        )
        .with_columns(
            pl_norm(db_cols).alias("dB_vec"),
        )
        .with_columns(
            (pl.col("dB_vec") / pl.col("B_mean")).alias("index_diff"),
        )
    )

    return index_diff

In [ ]:
# | export
def _compute_indices(
    df: pl.LazyFrame, tau: timedelta, cols: list[str] = ["BX", "BY", "BZ"]
) -> pl.LazyFrame:
    join_strategy = "inner"

    std_df = compute_std(df, tau, cols)
    combined_std_df = compute_combinded_std(df, tau, cols)

    index_std = compute_index_std(std_df, tau)
    index_diff = compute_index_diff(df, tau, cols)

    indices = (
        index_std.join(index_diff, on="time")
        .join(combined_std_df, on="time", how=join_strategy)
        .with_columns(
            pl.sum_horizontal("B_std_prev", "B_std_next").alias("B_added_std"),
        )
        .with_columns(
            (pl.col("B_std") / (pl.max_horizontal("B_std_prev", "B_std_next"))).alias(
                "index_std"
            ),
            (pl.col("B_combined_std") / pl.col("B_added_std")).alias(
                "index_fluctuation"
            ),
        )
    )

    return indices


def compute_indices(
    df: pl.DataFrame, tau: timedelta, bcols: list[str] = ["BX", "BY", "BZ"]
) -> pl.DataFrame:
    """
    Compute all index based on the given DataFrame and tau value.

    Parameters
    ----------
    df : pl.DataFrame
        Input DataFrame.
    tau : datetime.timedelta
        Time interval value.

    Returns
    -------
    tuple :
        Tuple containing DataFrame results for fluctuation index,
        standard deviation index, and 'index_num'.

    Examples
    --------
    >>> indices = compute_indices(df, tau)

    Notes
    -----
    - This is a wrapper for `_compute_indices` with `pl.LazyFrame` input.
    - Simply shift to calculate index_std would not work correctly if data is missing,
        like `std_next = pl.col("B_std").shift(-2)`.
    - Drop null though may lose some IDs (using the default `join_strategy`).
        Because we could not tell if it is a real ID or just a partial wave
        from incomplete data without previous or/and next std.
        Hopefully we can pick up the lost ones with smaller tau.
    - TODO: Can be optimized further, but this is already fast enough.
        - TEST: if `join` can be improved by shift after filling the missing values.
        - TEST: if `list` in `polars` really fast?
    """
    return _compute_indices(df.lazy(), tau, bcols).collect()